# Base Map
This downloads the DC street network

### <b> Install necessary packages
Check whether you have the necessary packages, and install

In [ ]:
#copy and paste into terminal
#geopandas
#python -m pip install geopandas
#open street maps
#python -m pip install osmnx
#matplot library
#python -m pip install matplotlib


### <b> Loading libraries </b>

In [ ]:
#Import packages
import os #deal with files and directories
import geopandas as gpd #handle geospatial data
import osmnx as ox #open streets package
import matplotlib.pyplot as plt #plotting
import folium #for our interactive base map

### <b> Configuring OSM </b>

In [ ]:
# Define the place name for OpenStreetMap
DMV_places = [
    "Washington, District of Columbia, USA",
    "Arlington County, Virginia, USA",
    "Alexandria, Virginia, USA",
    "Fairfax County, Virginia, USA",
    "Montgomery County, Maryland, USA",
    "Prince George's County, Maryland, USA"
]

# Define paths for saving data
# Using relative paths makes the code portable
raw_data_dir = "../data/raw"
os.makedirs(raw_data_dir, exist_ok=True) # Create the directory if it doesn't exist

streets_filename = os.path.join(raw_data_dir, "dmv_streets_raw.geojson")

### <b> Create raw GeoJSON </b>

In [ ]:
#Get data
print(f"Downloading street network for the DMV area...")
# Download the street network for all specified places.
# 'network_type="all"' gets all public streets.
# We use ox.utils_geo._geocode_to_gdf to get the geometries and then merge them
# This is more robust for multiple areas than graph_from_place
gdf_places = ox.geocode_to_gdf(DMV_places)
# Create a single unified polygon from all the place geometries
combined_polygon = gdf_places.unary_union

print("Downloading street network within the combined DMV polygon...")
# Download the network within the combined polygon
G = ox.graph_from_polygon(combined_polygon, network_type="all")

print("Converting graph to GeoDataFrame...")
# Convert the network graph into GeoDataFrames (one for nodes, one for edges/streets)
# We are interested in the streets (edges)
nodes, edges = ox.graph_to_gdfs(G)

print(f"Download complete. Found {len(edges)} street segments in the DMV area.")

### <b> Save GeoJSON

In [ ]:
# Cell 5: Save the Data
print(f"Saving raw street data to {streets_filename}...")
# Save the edges GeoDataFrame to a file
edges.to_file(streets_filename, driver="GeoJSON")

### <b>  Open GeoJSON (if previously saved) </b>

In [ ]:
# Load the data from the GeoJSON file into a GeoDataFrame
print(f"Loading street data from {streets_filename}...")
edges = gpd.read_file(streets_filename)

print("Data loaded successfully!")
print(f"The dataset contains {len(edges)} street segments.")
print("Here's a preview of the data:")
edges.head()

### <b> Visualize streets network only </b>

In [ ]:
# Create a simple plot to check the data
fig, ax = plt.subplots(figsize=(12, 12))
edges.plot(ax=ax, linewidth=0.5, color='gray')
ax.set_title("DMV Street Network - Raw Data")
ax.set_axis_off()
plt.show()

### <b> Visualize interactive base map </b>

In [ ]:
# Calculate the center of the combined DMV area for the map's initial view
#bounds = gdf_places.total_bounds
#center_lat = (bounds[1] + bounds[3]) / 2
#center_lon = (bounds[0] + bounds[2]) / 2

#Calculate center of DC specifically
DC_places = "Washington, District of Columbia, USA"
gdf_DCplaces = ox.geocode_to_gdf(DC_places)
bounds = gdf_DCplaces.total_bounds
center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2 

print(bounds)

In [ ]:
#Base map


# Create a folium map centered on the DMV or DCarea
#Options include 
#"OpenStreetMap": The default and widely used open-source map.
#"CartoDB Positron": A light-themed, clean map style.
#"CartoDB DarkMatter": A dark-themed map style.
#"Mapbox Bright": and "Mapbox Control Room": Mapbox tiles with limited zoom levels in their free versions.
m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles='OpenStreetMap')

display(m)

In [ ]:
#Sample data with 5% of streets
edges_sample = edges.sample(frac=0.05)

#Create a GeoDataFrame to a Folium map
# We also use a simple style function to make the lines gray and thin
street_geojson = folium.GeoJson(
    edges_sample,
    style_function=lambda feature: {
        'color': 'gray',
        'weight': 1.5,
        'opacity': 0.7
    }
).add_to(m)

display(m)

In [ ]:
# Add a layer control to toggle the street data on/off
folium.LayerControl().add_to(m)

# Display the map in the notebook
display(m)

# Also, save the map to an HTML file in the output directory
output_map_dir = "../output/maps"
os.makedirs(output_map_dir, exist_ok=True)
output_map_path = os.path.join(output_map_dir, "dmv_basemap_sample_interactive.html")
m.save(output_map_path)

print(f"Interactive base map (with sampled data) saved to {output_map_path}")
print("You can open this file in your web browser to explore the map.")

In [ ]:
# Add the street data to the map
# We iterate through each street segment in our GeoDataFrame
# This can be slow for very large datasets, but it's manageable for the DMV.
print("Adding street segments to the map (this may take a moment)...")
for _, row in edges.iterrows():
    # Check if the geometry is a LineString
    if row.geometry.geom_type == 'LineString':
        # Get the coordinates for the street
        coords = [[point[1], point[0]] for point in row.geometry.coords]
        
        # Add the street as a PolyLine to the map
        folium.PolyLine(
            locations=coords,
            color='gray',
            weight=1.5,
            opacity=0.7
        ).add_to(m)

# Add a layer control to toggle the street data on/off
folium.LayerControl().add_to(m)

# Display the map in the notebook
display(m)

# Also, save the map to an HTML file in the output directory
output_map_dir = "../output/maps"
os.makedirs(output_map_dir, exist_ok=True)
output_map_path = os.path.join(output_map_dir, "dmv_basemap_interactive.html")
m.save(output_map_path)

print(f"Interactive base map saved to {output_map_path}")
print("You can open this file in your web browser to explore the map.")